In [1]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json


c:\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
from pathlib import Path
import pinecone
from pprint import pprint

#from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

import google.generativeai as genai

In [3]:
PINECONE_API_KEY= "9ad6e8f9-0adb-4f03-b03b-2b94e650ecf0"

In [4]:
#model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
#"jinaai/jina-embeddings-v2-base-en", # switch to en/zh for English or Chinese
# trust_remote_code=True

model_name = SentenceTransformer(
    "intfloat/multilingual-e5-large", # switch to en/zh for English or Chinese
    trust_remote_code=True
)

#embeddings = HuggingFaceEmbeddings(model_name = model_name)

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Poornima\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
pc = Pinecone(api_key="9ad6e8f9-0adb-4f03-b03b-2b94e650ecf0")

In [6]:


dim = 1024

# Create a Pinecone index
pc.create_index(
    name="rag-index",
    dimension=dim,
    metric="cosine",
       spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
    

In [8]:
data = json.load(open("reviews.json"))

In [9]:
#create embedding for each review and store in vector db

processed_data = []
for reviews in data["professor_reviews"]: 
    #for review in reviews: 
        #embedding = embeddings.embed_query(reviews["comment"])
        embedding = model_name.encode(reviews["comment"])
        print(embedding)
        processed_data.append(
        {
            "values": embedding,
            "id": reviews["name"],
            "metadata":{
                "review": reviews["comment"],
                "subject": reviews["subject"],
                "stars": reviews["rating"],
            }
        }
        )

         

[ 0.02432381 -0.02364407 -0.02100378 ... -0.00223902 -0.02795023
  0.00475863]
[-0.00092918  0.01592753 -0.02330413 ... -0.03121607 -0.011764
 -0.0118153 ]
[ 0.0177086  -0.02911551  0.0022586  ... -0.00421246 -0.01322166
  0.02484181]
[ 0.00983218  0.00582714 -0.02218431 ... -0.04082119 -0.02098964
  0.00941287]
[ 0.01146369  0.01084676 -0.04239646 ... -0.04071549 -0.04683482
  0.02721784]
[ 0.02143234  0.00369757 -0.02597647 ... -0.02674247 -0.03878894
  0.00024299]
[ 0.0058188  -0.02052629 -0.02318107 ... -0.00289585 -0.01324174
  0.01334855]
[ 0.00435513  0.00102532 -0.02066496 ... -0.01982301 -0.01021237
  0.01662004]
[ 0.03244032 -0.01726412 -0.02438657 ... -0.01283096 -0.03496867
 -0.00700557]
[-0.00209978 -0.00979444 -0.02363471 ... -0.03463656 -0.02126561
 -0.00039217]
[ 0.00467627  0.00158913 -0.02305844 ... -0.01281487 -0.01914744
 -0.00092732]
[ 0.00342707  0.0022107  -0.03820909 ... -0.03097768 -0.04567344
  0.00622324]
[-0.00826303 -0.00509928 -0.02697211 ... -0.01670809 -

In [10]:
print(processed_data)

[{'values': array([ 0.02432381, -0.02364407, -0.02100378, ..., -0.00223902,
       -0.02795023,  0.00475863], dtype=float32), 'id': 'Dr. Sarah Johnson', 'metadata': {'review': 'Excellent at explaining complex concepts. Engaging lectures!', 'subject': 'Physics', 'stars': 4.5}}, {'values': array([-0.00092918,  0.01592753, -0.02330413, ..., -0.03121607,
       -0.011764  , -0.0118153 ], dtype=float32), 'id': 'Prof. Michael Lee', 'metadata': {'review': 'Knowledgeable, but assignments can be unclear.', 'subject': 'Literature', 'stars': 3.0}}, {'values': array([ 0.0177086 , -0.02911551,  0.0022586 , ..., -0.00421246,
       -0.01322166,  0.02484181], dtype=float32), 'id': 'Dr. Emily Chen', 'metadata': {'review': 'Fantastic teacher! Always available for extra help.', 'subject': 'Computer Science', 'stars': 4.8}}, {'values': array([ 0.00983218,  0.00582714, -0.02218431, ..., -0.04082119,
       -0.02098964,  0.00941287], dtype=float32), 'id': 'Prof. Robert Taylor', 'metadata': {'review': 'Lect

In [11]:
index = pc.Index("rag-index")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 25 Aug 2024 13:10:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1329', 'x-pinecone-request-id': '5359569843297793653', 'x-envoy-upstream-service-time': '53', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 1024 does not match the dimension of the index 768","details":[]}
